In [1]:
import os
import pandas as pd
import traceback
import json

In [2]:
from langchain_community.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env")  # This works because you're in the same folder as .env
key = os.getenv("OPENAI_API_KEY")
print(key)


None


In [4]:
import os
print(os.path.exists(".env"))  # Should print True

True


In [5]:
from dotenv import load_dotenv
import os

# Step 1: Load .env file
loaded = load_dotenv(dotenv_path=".env")
print("✅ load_dotenv() success:", loaded)

# Step 2: Check if .env exists in this folder
print("✅ .env exists:", os.path.exists(".env"))

# Step 3: Get the key
key = os.getenv("OPENAI_API_KEY")
print("✅ OPENAI_API_KEY:", key)


✅ load_dotenv() success: False
✅ .env exists: True
✅ OPENAI_API_KEY: None


In [7]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env")
print("✅ .env exists:", os.path.exists(".env"))
print("✅ KEY:", os.getenv("OPENAI_API_KEY"))


✅ .env exists: True
✅ KEY: None


In [5]:
llm=ChatOpenAI(temperature=1, model_name="gpt-3.5-turbo",openai_api_key=key)

C:\Users\baher\AppData\Local\Temp\ipykernel_36672\478087435.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(temperature=1, model_name="gpt-3.5-turbo",openai_api_key=key)


ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'temperature': 1, 'model...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
ChatOpenAI()